In [1]:
import csv
import pandas as pd
import string
import requests
from xml.dom import minidom
import nltk
from bs4 import BeautifulSoup

In [2]:
import xml.etree.ElementTree as ET

def xml2df(path):
    tree = ET.parse(path)
    root = tree.getroot()
    data=[]
    for child in root:
        #print(child.attrib)
        #df = pd.Series(child.attrib).to_frame()
        data.append(child.attrib)
    df = pd.DataFrame(data)
    return df

In [3]:
def create_data(category):
    
#     df_post= xml2df("/home/rachneet/Downloads/stack_exchange/" + category + ".stackexchange.com/Posts.xml")
#     df_user= xml2df("/home/rachneet/Downloads/stack_exchange/" + category + ".stackexchange.com/Users.xml")
    
    df_post= xml2df("/home/rachneet/datasets/stack_overflow/Posts.xml")
    df_user= xml2df("/home/rachneet/datasets/stack_overflow/Users.xml")
    
    print("Posts: ",df_post.shape)
    df_post = df_post[['OwnerUserId','Body']]
    print("Users: ",df_user.shape)
    df_user = df_user[['Id','DisplayName']]
    df_post.rename(columns={'OwnerUserId':'Id'},inplace=True)
    data = pd.merge(df_post,df_user,on='Id',how='left')
    data.dropna(inplace=True)
    print("Combined frame shape", data.shape)
    data.drop(['Id'],axis=1,inplace=True)
    data.rename(columns={'DisplayName':'Name','Body':'Post'},inplace=True)
    data = data.reindex(columns=['Name','Post'])
    data['Post'] = data['Post'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().replace('|','').lower())
    data['Post'] = data['Post'].apply(lambda x: " ".join(x.split()))
#     soup = BeautifulSoup(data.at[0,'Post'], 'html.parser')
#     print(soup.get_text())
    #print(data.head())
    #print(data.at[1,'Post'])
    #data.to_csv("/home/rachneet/datasets/stack_overflow/" + category + ".csv",sep='|',encoding = 'utf-8',header=False,index=False,na_rep=string)
    print("Write complete for post: ", category)
    return data

In [4]:
df = create_data('stack_overflow_dataset')

Posts:  (43872992, 21)
Users:  (10097978, 13)
Combined frame shape (43384834, 3)


/home/rachneet/.local/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://en.wikipedia.org/wiki/Arity#n-ary" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/rachneet/.local/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://en.wikipedia.org/wiki/CPUID" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/rachneet/.local/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "https://github.com/davedelong/DDMathParser" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the

Write complete for post:  stack_overflow_dataset


In [5]:
df.head()

,Name,Post
0,Eggs McLaren,i want to use a track-bar to change a form's o...
1,Kevin Dente,i have an absolutely positioned div containing...
2,Kevin Dente,an explicit cast to double like this isn't nec...
3,Jeff Atwood,given a datetime representing a person's birth...
4,Jeff Atwood,"given a specific datetime value, how do i disp..."


In [6]:
df.shape

(43384834, 2)

In [7]:
#UnicodeEncodeError: 'utf-8' codec can't encode character '\udbff' in position 789: surrogates not allowed

In [8]:
def cleanser(text):
    post=''
    try:
        post=  text.encode("utf-8")
    except UnicodeEncodeError as e:
        if e.reason == 'surrogates not allowed':
            post =  text.encode('utf-8', "backslashreplace").decode('utf-8')
    
    return post

In [12]:
df['Post'] = df['Post'].apply(lambda x: cleanser(x))

In [13]:
df.shape

(43384834, 2)

In [11]:
# x= "i love this"
# print(x.encode('utf-8').decode('utf-8'))
# print(x)

i love this
i love this


In [14]:
df.head()

,Name,Post
0,Eggs McLaren,"b""i want to use a track-bar to change a form's..."
1,Kevin Dente,"b""i have an absolutely positioned div containi..."
2,Kevin Dente,"b""an explicit cast to double like this isn't n..."
3,Jeff Atwood,"b""given a datetime representing a person's bir..."
4,Jeff Atwood,"b'given a specific datetime value, how do i di..."


In [15]:
df.to_csv("/home/rachneet/datasets/stack_overflow/stack_overflow_dataset.csv",sep='|',header=False,index=False,na_rep=string)

In [19]:
def clean(text):
    post =''
    try:
        post = text.decode('utf-8')
    except AttributeError:
        post =  text
    return post

In [20]:
df['Post'] = df['Post'].apply(lambda x: clean(x))

In [21]:
df.head()

,Name,Post
0,Eggs McLaren,i want to use a track-bar to change a form's o...
1,Kevin Dente,i have an absolutely positioned div containing...
2,Kevin Dente,an explicit cast to double like this isn't nec...
3,Jeff Atwood,given a datetime representing a person's birth...
4,Jeff Atwood,"given a specific datetime value, how do i disp..."


In [22]:
df.to_csv("/home/rachneet/datasets/stack_overflow/stack_overflow_set.csv",sep='|',encoding='utf-8',header=False,index=False,na_rep=string)

In [23]:
df.shape

(43384834, 2)

# gender guesser 

In [4]:
cols = ['Name','Post']
df = pd.read_csv('/home/rachneet/datasets/stack_overflow/stack_overflow_set.csv', sep='|',encoding='utf8',quoting=csv.QUOTE_NONE,names=cols)

In [6]:
df.head()

,Name,Post
0,Eggs McLaren,i want to use a track-bar to change a form's o...
1,Kevin Dente,i have an absolutely positioned div containing...
2,Kevin Dente,an explicit cast to double like this isn't nec...
3,Jeff Atwood,given a datetime representing a person's birth...
4,Jeff Atwood,"given a specific datetime value, how do i disp..."


In [7]:
import gender_guesser.detector as gender
d = gender.Detector(case_sensitive=False)

In [8]:
translator = str.maketrans('', '', string.punctuation)

In [13]:
def get_gender(name):
    try:
        n = name.lower()
        if ('mom' in n) or ('girl' in n) or ('angel' in n) or ('mum' in n) or ('mother' in n) or ('woman' in n):
            return 'female'
        if ('boy' in n) or ('dude' in n):
            return 'male'
        temp = name.translate(translator).split()
    except:
        return 'unknown'
    
    if len(temp)>0:
        first_name = temp[0]
        first_name = ''.join([i for i in first_name if not i.isdigit()])
        gender = d.get_gender(first_name,'usa')
        if(gender=='unknown'):
            first_name= first_name[:-1]
            gender = d.get_gender(first_name,'usa')
        return gender
    else:
        return 'unknown'

In [19]:
df['Gender'] = df['Name'].apply(lambda x:get_gender(x))

In [20]:
df.Gender.value_counts()

unknown          22800536
andy              9935150
male              8409803
mostly_male       1378369
female             616065
mostly_female      244911
Name: Gender, dtype: int64

In [21]:
df.head()

,Name,Post,Gender
0,Eggs McLaren,i want to use a track-bar to change a form's o...,unknown
1,Kevin Dente,i have an absolutely positioned div containing...,male
2,Kevin Dente,an explicit cast to double like this isn't nec...,male
3,Jeff Atwood,given a datetime representing a person's birth...,male
4,Jeff Atwood,"given a specific datetime value, how do i disp...",male


In [25]:
df_male = df.loc[df.Gender=='male']

In [26]:
df_male.head()

,Name,Post,Gender
1,Kevin Dente,i have an absolutely positioned div containing...,male
2,Kevin Dente,an explicit cast to double like this isn't nec...,male
3,Jeff Atwood,given a datetime representing a person's birth...,male
4,Jeff Atwood,"given a specific datetime value, how do i disp...",male
5,Jeff Atwood,"""here's how i do it var ts = new timespan(date...",male


In [27]:
df_female=df.loc[df.Gender=='female']

In [28]:
df_female.head()

,Name,Post,Gender
138,Kalid,"i often have to sort a dictionary, consisting ...",female
140,Kalid,"looking around, and using some c# 3.0 features...",female
164,Kalid,"""in ruby on rails, there's a concept of a migr...",female
1272,Lea Cohen,if you want to teach the basics of programming...,female
1657,Lily Ballard,you should take a look at git instead of svn. ...,female


In [29]:
df_female.shape

(616065, 3)

In [32]:
df_male_reduced = df_male.sample(n=616065)

In [33]:
df_male_reduced.head()

,Name,Post,Gender
11099893,Joseph Mansfield,"""that's what the carriage return character is ...",male
5665370,Hank,since i have no control of which connections a...,male
25174981,Quentin Hayot,if you really need to download your data every...,male
5952974,James Dunay,"""i can't figure out whats happening here and w...",male
6189870,david,"""how do you replace all non matches from one a...",male


In [34]:
df_male_reduced.shape

(616065, 3)

In [35]:
data = df_male_reduced.append(df_female, ignore_index=True)

In [36]:
data.shape

(1232130, 3)

In [37]:
data.head()

,Name,Post,Gender
0,Joseph Mansfield,"""that's what the carriage return character is ...",male
1,Hank,since i have no control of which connections a...,male
2,Quentin Hayot,if you really need to download your data every...,male
3,James Dunay,"""i can't figure out whats happening here and w...",male
4,david,"""how do you replace all non matches from one a...",male


In [38]:
df2 = data.groupby('Name').agg({'Post':'count','Gender':'first'}).reset_index().rename(columns={'Post':'Count'})

In [39]:
df2.head()

,Name,Count,Gender
0,Maria,1,female
1,Niki Hard,1,female
2,Allen,1,male
3,Anna,2,female
4,Benny Duikelaar,1,male


In [41]:
df_m = df2.loc[df2.Gender=='male']
df_f = df2.loc[df2.Gender=='female']
print(df_m.shape,df_f.shape)

(107654, 3) (54061, 3)


In [42]:
train_ids =[]
test_ids=[]
count=0
c_tr = 0
c_ts=0
limit = round(.75*df_m.Count.sum())
for index, row in df_m.iterrows():
    count+= row['Count']
    
    if(count<= limit):
        train_ids.append(row['Name'])
        c_tr+= row['Count']
    else:
        test_ids.append(row['Name'])
        c_ts+= row['Count']

print(len(train_ids),c_tr)
print(len(test_ids),c_ts)

79871 461248
27783 154740


In [43]:
count=0
c_tr = 0
c_ts=0
limit = round(.75*df_f.Count.sum())
for index, row in df_f.iterrows():
    count+= row['Count']
    
    if(count<= limit):
        train_ids.append(row['Name'])
        c_tr+= row['Count']
    else:
        test_ids.append(row['Name'])
        c_ts+= row['Count']

print(len(train_ids),c_tr)
print(len(test_ids),c_ts)

121888 462017
39827 154040


In [44]:
df_train = data.loc[data.Name.isin(train_ids)].reset_index()

In [45]:
df_train.head()

,index,Name,Post,Gender
0,0,Joseph Mansfield,"""that's what the carriage return character is ...",male
1,1,Hank,since i have no control of which connections a...,male
2,2,Quentin Hayot,if you really need to download your data every...,male
3,3,James Dunay,"""i can't figure out whats happening here and w...",male
4,5,Brian Ball,there are already some good suggestions. i wou...,male


In [47]:
df_train.shape

(923341, 4)

In [48]:
df_test = data.loc[data.Name.isin(test_ids)].reset_index()

In [49]:
df_test.shape

(308789, 4)

In [50]:
df_train.drop(['index'],axis=1, inplace=True)
df_test.drop(['index'],axis=1, inplace=True)

In [51]:
df_train.drop(['Name'],axis=1, inplace=True)

In [52]:
df_train['Label'] = pd.get_dummies(df_train.Gender)['female']
df_train.drop(['Gender'],axis=1, inplace=True)
df_train.head()

,Post,Label
0,"""that's what the carriage return character is ...",0
1,since i have no control of which connections a...,0
2,if you really need to download your data every...,0
3,"""i can't figure out whats happening here and w...",0
4,there are already some good suggestions. i wou...,0


In [53]:
df_train = df_train[['Label','Post']]

In [54]:
df_train.head()

,Label,Post
0,0,"""that's what the carriage return character is ..."
1,0,since i have no control of which connections a...
2,0,if you really need to download your data every...
3,0,"""i can't figure out whats happening here and w..."
4,0,there are already some good suggestions. i wou...


In [55]:
df_train = df_train.sample(frac = 1).reset_index(drop=True)
df_train.head()

,Label,Post
0,0,you should store the fact that that a row as b...
1,0,django 1.9.7 postgresql could you help me unde...
2,1,"""we have a lot of ajax calls in our applicatio..."
3,0,"""i am trying to implement a program that will ..."
4,1,"when an element is fixed, it needs its coordin..."


In [56]:
df_train.to_csv("/home/rachneet/datasets/stack_overflow/train_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)

In [57]:
df_test['Label'] = pd.get_dummies(df_test.Gender)['female']
df_test.drop(['Gender'],axis=1, inplace=True)
df_test = df_test[['Name','Label','Post']]
df_test = df_test.sample(frac = 1).reset_index(drop=True)
df_test.head()

,Name,Label,Post
0,Zack Macomber,0,"""i have the following scope function in my con..."
1,Tim,0,"file.readlines returns an ienumerable<string>,..."
2,Xander Angell,1,i am working in html 5 and wanted to add an im...
3,Trevor Hickey,0,"""i understand the motivation for using std::st..."
4,murielg,1,"""i am trying to append a string, myoutput, rig..."


In [58]:
x = round(df_test.shape[0] * 0.80)
test_set = df_test.iloc[:x]
val_set = df_test.iloc[x:]
print(test_set.shape, val_set.shape)

(247031, 3) (61758, 3)


In [59]:
test_set.head()

,Name,Label,Post
0,Zack Macomber,0,"""i have the following scope function in my con..."
1,Tim,0,"file.readlines returns an ienumerable<string>,..."
2,Xander Angell,1,i am working in html 5 and wanted to add an im...
3,Trevor Hickey,0,"""i understand the motivation for using std::st..."
4,murielg,1,"""i am trying to append a string, myoutput, rig..."


In [60]:
test_set.to_csv("/home/rachneet/datasets/stack_overflow/test_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)

In [61]:
val_set.drop(['Name'],axis=1, inplace=True)
val_set.head()

/home/rachneet/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Label,Post
247031,0,"""i did it shown in this example: http://blog.s..."
247032,0,"""please read the comments to this post underst..."
247033,0,"""i'm trying to hide/show an action bar button ..."
247034,1,i'm wondering where are gcc macros like builti...
247035,1,i am looking for a good wiki software to host ...


In [62]:
val_set.to_csv("/home/rachneet/datasets/stack_overflow/validation_so.csv",encoding = 'utf-8',header=False,index=False,na_rep=string)